In [ ]:
import requests
import datetime
import random
from geopy.geocoders import Nominatim
import sqlite3

In [ ]:
conn = sqlite3.connect('wb.db')
cursor = conn.cursor()

In [ ]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS positions (
        name TEXT,
        cpm integer,
        position integer,
        promoPosition integer,
        tp TEXT,
        query TEXT,
        created_at datetime,
        city TEXT
    )
''')
conn.commit()

In [ ]:
cursor.execute('select * from positions')
cursor.fetchall()

[]

In [ ]:
query_lst = ['антифриз', 'антифриз зеленый', 'антифриз красный']
max_page = 2
brand = 'SINTEC'

search_arr = []

In [ ]:
city_lst = ['Москва', 'Санкт-Петербург', 'Казань', 'Красноярск']

def generate_city_params(city):
  geolocator = Nominatim(user_agent="my_app")
  location = geolocator.geocode(city)
  params = {
      'latitude': location.latitude,
      'longitude': location.longitude,
      'address': city
  }
  city_params = requests.get('https://user-geo-data.wildberries.ru/get-geo-info', params = params)
  return city_params

In [ ]:
for city in city_lst:

  print(f'\n\n!!! {city} !!!')

  geo_info = generate_city_params(city)

  for query in query_lst:
    print(f'\n\n\nЗапрос: {query}\n')

    cnt = 0

    for page in range(1, max_page + 1):

        print(f'СТРАНИЦА {page}')

        res = requests.get(f'''
          https://search.wb.ru/exactmatch/ru/common/v13/search?
          ab_testing=false&
          lang=ru&
          page={page}&
          query={query}&
          resultset=catalog&
          sort=popular&
          suppressSpellcheck=false&
          {geo_info.json()['xinfo']}
        '''.replace('\n', '').replace(' ', '').strip())

        products = res.json()['data']['products']

        for product in products:

            cnt += 1 # cnt = cnt + 1

            if product['brand'] == brand:
                print(product['name'])

                # datetime.datetime.now()
                start_date = datetime.datetime(2025, 1, 1)
                finish_date = datetime.datetime(2026, 12, 31)
                diff = (finish_date - start_date).total_seconds()
                random_seconds = random.randint(0, int(diff))
                random_dt = start_date + datetime.timedelta(seconds = random_seconds)

                if product.get('log'):
                  search_arr.append([
                      product['name'],
                      product['log']['cpm'],
                      product['log']['position'],
                      product['log']['promoPosition'],
                      product['log']['tp'],
                      query,
                      random_dt,
                      city
                  ])
                else:
                  search_arr.append([
                      product['name'],
                      0,
                      cnt,
                      -1,
                      '-',
                      query,
                      random_dt,
                      city
                  ])



!!! Москва !!!





Запрос: антифриз

СТРАНИЦА 1
Антифриз EURO G11 (-45°С) зеленый, силикатный 5кг
Антифриз EURO G11 (-40°С) силикатный, зеленый 5кг
Антифриз LUXE G12+ (-45°С) карбоксилатный, красный 5кг
Антифриз G11 зеленый
Антифриз готовый фиолетовый Синтек MULTIFREEZE 5кг
Антифриз G11 зеленый
Антифриз G11 зеленый
Антифриз Antifreeze OEM China OAT red -40 5кг
Антифриз MULTIFREEZE -40 лобридный, фиолетовый 5кг
Антифриз G11 зеленый
СТРАНИЦА 2
Антифриз G11 зеленый
Антифриз G11 зеленый



Запрос: антифриз зеленый

СТРАНИЦА 1
Антифриз EURO G11 (-40°С) силикатный, зеленый 5кг
Антифриз EURO G11 (-45°С) зеленый, силикатный 5кг
Антифриз G11 зеленый
Антифриз G11 зеленый
Антифриз G11 зеленый
Антифриз G11 зеленый
Антифриз G11 зеленый
Антифриз G11 зеленый
Антифриз EURO G11 зеленый -40 Акция 5кг по цене 4кг
G11 EURO антифриз green -40 5кг
G11 EURO антифриз green -40 10кг
G11 EURO green -40 1кг
Антифриз зеленый G11 Синтек EURO 5кг
Антифриз EURO G11 (-40) силикатный, зеленый, 5 кг по цене 4
СТРАНИЦА 2
Антифриз Синте

In [ ]:
for row in search_arr:
  cursor.execute('''
    insert into positions values (?, ?, ?, ?, ?, ?, ?, ?)
  ''', row)
  conn.commit()

In [ ]:
cursor.execute('select count(*) from positions')
cursor.fetchall()

[(415,)]

In [ ]:
import pandas as pd

df = pd.DataFrame(search_arr)

df.columns = ['name', 'cpm', 'pos', 'promopos', 'tp', 'query', 'dt', 'city']

df['year'] = df['dt'].dt.year

df.groupby(['name', 'query', 'year', 'city'])[['pos', 'promopos']].mean()

pos  \
name                               query            year city                     
G11 EURO green -40 1кг             антифриз зеленый 2025 Казань            81.0   
                                                         Москва            89.0   
                                                    2026 Красноярск        84.0   
                                                         Санкт-Петербург   90.0   
G11 EURO антифриз green -40 10кг   антифриз зеленый 2025 Красноярск        69.0   
...                                                                         ...   
Антифриз красный Luxe G12+, 5кг    антифриз красный 2026 Санкт-Петербург  199.0   
Антифриз красный OEM China -40 5кг антифриз красный 2025 Казань            36.0   
                                                         Москва            37.0   
                                                         Санкт-Петербург   34.5   
                                                    2026 Красноярск        34.5   

                                                                          promopos  
name                               query            year city                       
G11 EURO green -40 1кг             антифриз зеленый 2025 Казань               -1.0  
                                                         Москва               -1.0  
                                                    2026 Красноярск           -1.0  
                                                         Санкт-Петербург      -1.0  
G11 EURO антифриз green -40 10кг   антифриз зеленый 2025 Красноярск           -1.0  
...                                                                            ...  
Антифриз красный Luxe G12+, 5кг    антифриз красный 2026 Санкт-Петербург      -1.0  
Антифриз красный OEM China -40 5кг антифриз красный 2025 Казань               -1.0  
                                                         Москва               -1.0  
                                                         Санкт-Петербург      -1.0  
                                                    2026 Красноярск           -1.0  

[148 rows x 2 columns]